In [1]:
%pip -q install opencv-python pyyaml

Note: you may need to restart the kernel to use updated packages.


### **Azure AI Services**

Login to Azure using the Azure portal and retrieve your service endpoint and key.

- Open Microsoft Edge using the desktop shortcut and open Azure portal using the bookmark.

- Login using the Azure credentials in the Resources tab of the lab guide.

- Inside Azure portal, search for Azure AI Services and select it.

    ![A screenshot showing Azure AI services being searched for in the dropdown](../labguide/img/azureportal_ai_search.png)

- Note the name of the Azure AI service that was precreated for you and then click on the resource.

    ![A screenshot showing an Azure AI service inside a resource group in Azure portal](../labguide/img/azure_ai_service.png)

- You will need the AI service name, language endpoint and one of the key values for the Azure AI Service. You will need these values in the next cell. Review the screenshot and locate these values in your Azure environment, then replace the placeholders in the next cell with your values.

    1. Service name
    2. Key (pick either key 1 or key 2)
    3. Language endpoint

    ![A screenshot showing an Azure AI service overview page in Azure portal](../labguide/img/ai_endpoint_key.png)



### Define some helper functions

We will define two functions to assist with invoking the gpt-4o model's API.

In [2]:
import os, requests, base64
from pprint import pprint

# Call the AI service API
def call_api(uri, key, payload):
    headers = {
        "Content-Type": "application/json",
        "api-key": key,
    }
    # Send request
    try:
        response = requests.post(uri, headers=headers, json=payload)
        response.raise_for_status()  # Will raise an HTTPError if the HTTP request returned an unsuccessful status code
    except requests.RequestException as e:
        raise SystemExit(f"Failed to make the request. Error: {e}")

    # Handle the response as needed (e.g., print or process)
    response_json = response.json()
    return response_json

# Convert an image to base64 encoding
def image_to_base64(image_path_or_url):
    if image_path_or_url.startswith('http://') or image_path_or_url.startswith('https://'):
        response = requests.get(image_path_or_url)
        image_data = response.content
    else:
        with open(image_path_or_url, "rb") as image_file:
            image_data = image_file.read()
    return base64.b64encode(image_data).decode('utf-8')


### Configure with your Azure AI service endpoint and key

Replace the placeholders in the code block below endpoint and key you retrieved.

1. *azureAiServiceName*
2. *API_KEY*
3. *serviceUri*


In [7]:
# Replace these values with the correct values from your Azure AI service
azureAiServiceName = "testaisvc" # 1 - Service name
API_KEY = "4z2EFmQ6tV9CJMZagpv1Yr2LvZXPoeGU9rTA01BB1Bo2G3QZEFjqJQQJ99BAACYeBjFXJ3w3AAAAACOGLpzK" # 2 - API Key
serviceUri = "https://testaisvc.openai.azure.com/" # 3 - service endpoint

IMAGE_PATH="../media/image/people_on_street.jpg" # 4 - Image path
ENDPOINT = f"{serviceUri}openai/deployments/{azureAiServiceName}-gpt-4o-mini-deployment/chat/completions?api-version=2024-08-01-preview"

pprint(ENDPOINT)
ENDPOINT="https://testaisvc.cognitiveservices.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-08-01-preview"
#https://testaisvc.cognitiveservices.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-08-01-preview

'https://testaisvc.openai.azure.com/openai/deployments/testaisvc-gpt-4o-mini-deployment/chat/completions?api-version=2024-08-01-preview'


### Initialize the model with a prompt

Now we can initialize gpt-4o with a prompt. 

In [28]:
# Payload for the request
payload = {
  "messages": [
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "You are an AI assistant that helps people find information."
        }
      ]
    }
  ],
  "temperature": 0.7,
  "top_p": 0.95,
  "max_tokens": 800
}

call_api(ENDPOINT, API_KEY, payload)

{'choices': [{'content_filter_results': {'hate': {'filtered': False,
     'severity': 'safe'},
    'protected_material_code': {'filtered': False, 'detected': False},
    'protected_material_text': {'filtered': False, 'detected': False},
    'self_harm': {'filtered': False, 'severity': 'safe'},
    'sexual': {'filtered': False, 'severity': 'safe'},
    'violence': {'filtered': False, 'severity': 'safe'}},
   'finish_reason': 'stop',
   'index': 0,
   'logprobs': None,
   'message': {'content': 'How can I assist you today?',
    'refusal': None,
    'role': 'assistant'}}],
 'created': 1738162538,
 'id': 'chatcmpl-Av3la6S9HlE7YyjJUfyAGUvvjv8FF',
 'model': 'gpt-4o-mini-2024-07-18',
 'object': 'chat.completion',
 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {}}],
 'system_fingerprint': 'fp_5154047bf2',
 'usage': {'completion_tokens': 7,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_pre

### Describe an image with a prompt

To send an image with our prompt we will convert it to base64 and include it in the payload of our API call.

In [32]:
# Replace IMAGE_PATH with the path to the image you want to describe, or use the provided sample image.
IMAGE_PATH = "../media/image/columns.png"
image_base64 = image_to_base64(IMAGE_PATH)
payload = {
  "messages": [
    { "role": "system", "content": "You are a helpful assistant." }, # provide context for the model to generate a response
        { "role": "user", "content": [  
            { 
                "type": "text", 
                "text": "Describe this picture:"  # tell the model what to do
            },
            { 
                "type": "image_url",
                "image_url": {
                    "url": "data:image/png;base64," + image_base64 # send the base64 encoded image with the payload
                }
            }
        ] } 
  ],
  "temperature": 0.7,
  "top_p": 0.95,
  "max_tokens": 800
}

pprint(call_api(ENDPOINT, API_KEY, payload))

SystemExit: Failed to make the request. Error: 429 Client Error: Too Many Requests for url: https://testaisvc.cognitiveservices.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-08-01-preview

### Try other images. 

You can use any public URI as the IMAGE_PATH. Try uncommenting the samples below to try out different images and settings. 

You can also change the "role" prompt to get gpt-4o to respond more specifically.

In [37]:
# Replace IMAGE_PATH with the path to the image you want to describe, or use the provided sample image.
IMAGE_PATH = "https://c7.alamy.com/comp/2RFM9CB/closeup-of-acrylonitrile-butadiene-gloves-production-line-2RFM9CB.jpg"
#IMAGE_PATH = "https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcS3SyqcyXM0W3Sx3Q0DKPVkBu9GlxN7b06TQg&s"
image_base64 = image_to_base64(IMAGE_PATH)
payload = {
  "messages": [
    { "role": "system", "content": "You are a helpful assistant." }, # provide context for the model to generate a response
        { "role": "user", "content": [  
            { 
                "type": "text", 
                "text": "Describe this picture:"  # tell the model what to do
            },
            { 
                "type": "image_url",
                "image_url": {
                    "url": "data:image/png;base64," + image_base64 # send the base64 encoded image with the payload
                }
            }
        ] } 
  ],
  "temperature": 0.7,
  "top_p": 0.95,
  "max_tokens": 800
}

pprint(call_api(ENDPOINT, API_KEY, payload))

SystemExit: Failed to make the request. Error: 429 Client Error: Too Many Requests for url: https://testaisvc.cognitiveservices.azure.com/openai/deployments/gpt-4o-mini/chat/completions?api-version=2024-08-01-preview

## Complete Lab
Run the following cell to complete this lab!

In [ ]:
%store -r userId
import requests;print(requests.post("https://jsleaderboard001-cnece0effvapgbft.westus2-01.azurewebsites.net/complete_task", headers={"Content-Type": "application/json"}, json={"user_id": userId, "task_id": 5}).json())

### Continue

[Notebook 6 - Camera](./6-Camera.ipynb)